Se utiliza un método de filtrado colaborativo basado en usuarios.

# Inicialización de datos

Se abren los archivos y se guardan en memoria para poder hacer uso posterior de estos.

En esta sección se genera una matriz de ratings, pero dada la naturaleza _sparse_ de esta matriz se almacena en un diccionario de diccionarios con el siguiente formato

~~~
{ used_id : {
         movie_id: rating,
         },
  user_id2: {
         movie_id: rating,
         movie_id2: rating
         }
 }
    
~~~

In [1]:
movies_file = open('movies.dat', 'r', encoding="utf8")
ratings_file = open('ratings.dat', 'r', encoding="utf8")

matrix = dict()
users_matrix = dict()

In [2]:
def insert_sorted(a, tuple_value):
    _, value, count = tuple_value
    hi, lo = len(a), 0
    while lo < hi:
        mid = (lo + hi) // 2
        if value > a[mid][1] or (value == a[mid][1] and count > a[mid][2]):
            hi = mid
        else:
            lo = mid + 1
    a.insert(lo, tuple_value)

In [3]:
for line in movies_file:
    movie_id, name, tags = line.strip().split('::')
    matrix[movie_id] = {
        'name': name,
        'tags': tags.split('|'),
        'ratings': dict(),
        'ratings_count': 0,
        'total_rating': 0,
    }

In [4]:
for line in ratings_file:
    user_id, movie_id, rating, _ = line.strip().split('::')
    if matrix[movie_id]['ratings_count'] == 0:
        matrix[movie_id]['total_rating'] = float(rating)
    else:
        matrix[movie_id]['total_rating'] = round((
            float(rating) + (matrix[movie_id]['total_rating'] * matrix[movie_id]['ratings_count'])
        ) / (matrix[movie_id]['ratings_count'] + 1), 1)
    matrix[movie_id]['ratings'][user_id] = float(rating)
    matrix[movie_id]['ratings_count'] += 1
    
    if user_id not in users_matrix:
        users_matrix[user_id] = dict()
    users_matrix[user_id][movie_id] = float(rating)    

Ademas, se calculan las 10 películas más populares, basado en la cantidad de ratings que tiene cada una.

In [5]:
most_popular_movies = []
for movie_id, movie in matrix.items():
    insert_sorted(most_popular_movies, (movie_id, movie['ratings_count'], movie['total_rating']))

# Solicitar Ratings

Se guardan los ratings de las 10 películas más populares y se almacenan en la matriz de rating utilizando la clave _new_user_.

In [18]:
print('A continuación deberás puntuar 10 peliculas de acuerdo a tus gustos.')
print('Tu puntiación debe estar entre 0.5 y 5.0, en intervalos de 0.5, si no deseas opinar sobre una pelicula marca 0')

users_matrix['new_user'] = dict()

for movie_id, _, _ in most_popular_movies[:10]:
    ask_rating = True
    while ask_rating:
        try:
            rating = float(input('{}: '.format(matrix[movie_id]['name'])))
            
            if rating % 0.5 != 0 or rating < 0 or rating > 5:
                raise ValueError()
            
            if rating != 0:
                users_matrix['new_user'][movie_id] = rating 
            ask_rating = False
        except ValueError:
            print('Has ingresado de manera incorrecta el rating, recuerda que debe ser múltiplo de 0.5 y estar entre 0.5 y 5.0')

A continuación deberás puntuar 10 peliculas de acuerdo a tus gustos.
Tu puntiación debe estar entre 0.5 y 5.0, en intervalos de 0.5, si no deseas opinar sobre una pelicula marca 0
Pulp Fiction (1994): 5
Forrest Gump (1994): 4
Silence of the Lambs, The (1991): 3
Jurassic Park (1993): 2
Shawshank Redemption, The (1994): 1
Braveheart (1995): 2
Fugitive, The (1993): 3
Terminator 2: Judgment Day (1991): 4
Star Wars: Episode IV - A New Hope (a.k.a. Star Wars) (1977): 5
Apollo 13 (1995): 4


Para calcular similaridades se utiliza la similaridad de coseno, definida de la siguiente manera:

$$ sim(x, y) = \frac{<x,y>}{||x||_{2} \cdot ||y||_{2}}$$

In [19]:
def cosine_dist(user_1, user_2):
    dist = 0
    norm_user_1 = 0
    norm_user_2 = 0
    for movie_id, rating in users_matrix[user_1].items():
        norm_user_1 += rating ** 2
    for movie_id, rating in users_matrix[user_2].items():
        norm_user_2 += rating ** 2
        if movie_id in users_matrix[user_1]:
            dist += rating * users_matrix[user_1][movie_id]
    return dist/((norm_user_1 ** (1/2)) * (norm_user_2 ** (1/2)))

    

Se procede a calcular del nuevo usuario al resto de ellos utilizando la similaridad antes mecionada.

In [20]:
def get_dist_users():
    distances = []
    for user in users_matrix.keys():
        if user == 'new_user':
            continue
        distance = cosine_dist('new_user', user)
        distances.append((user,distance))
    return sorted(distances, key=lambda tup:tup[1], reverse=True)
        

In [21]:
distances_lists = get_dist_users()

Luego, se obtiene el vecindario del nuevo usuario. En este caso de utilizó un vecindario $K = 3$.

In [22]:
neighborhood = distances_lists[:3]

Finalmente se predice un rating para las películas que no ha visto el nuevo usuario, basadas en las películas que han visto los usuarios de su vencindario.

In [23]:
ranking = dict()
ready = []
for user, distance in neighborhood:
    for movie_id in users_matrix[user]:
        if movie_id not in users_matrix['new_user']:
            if movie_id not in ready:
                #print(movie_id)
                ranking[movie_id] = {
                    'num':0,
                    'dem':0
                }
                ready.append(movie_id)
            #print(ready, ranking)
            ranking[movie_id]['num'] += users_matrix[user][movie_id] * distance
            ranking[movie_id]['dem'] += distance

In [25]:
recomendation = []
for movie_id in ranking:
    recomendation.append((movie_id, ranking[movie_id]['num'] / ranking[movie_id]['dem']))

In [26]:
recomendation = sorted(recomendation,key= lambda tup:tup[1], reverse=True)

# Recomendaciones

In [28]:
for i in range(5):
    movie_id, rating = recomendation[i]
    print('{} - {} {}'.format(i + 1, matrix[movie_id]['name'], rating))

1 - Lion King, The (1994) 5.0
2 - Schindler's List (1993) 4.33064793873688
3 - Toy Story (1995) 4.33064793873688
4 - 12 Monkeys (Twelve Monkeys) (1995) 3.999080398722667
5 - Dances with Wolves (1990) 3.6631350800284266
